In [3]:
# Load model directly
from huggingface_hub import snapshot_download

snapshot_download(repo_id="Qwen/Qwen3-Embedding-8B", cache_dir='/data/LLMs/')

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

'/data/LLMs/models--Qwen--Qwen3-Embedding-8B/snapshots/a3d38e32b9c835d5b3d0d0a3ef3c133bbea92539'